# 04_previsao_ultrafast_parquet (⚡ Ultra Rápido)
Janela reduzida (N meses), 1 fold, calendário + lag1, Ridge & LightGBM, salva pipeline vencedor.

In [ ]:

# 1) IMPORTS & CONFIG
import os, json, warnings
from pathlib import Path
import numpy as np
import pandas as pd
warnings.filterwarnings("ignore")
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
try:
    from lightgbm import LGBMRegressor
except Exception:
    import sys, subprocess
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", "lightgbm"], check=False)
    from lightgbm import LGBMRegressor
import joblib

OUT_DIR = Path("./outputs"); OUT_DIR.mkdir(parents=True, exist_ok=True)
DEFAULT_PARQUET = os.environ.get("INPUT_FILE", r"/mnt/data/base_mensal_with_clusters (2).parquet")
DATE_COL, TARGET_COL, GRAIN_KEYS = "year_month", "sales", ["store","item"]
SEED, N_THREADS, MAX_MONTHS = 42, 2, int(os.environ.get("MAX_MONTHS", 18))
USE_LAG1 = True
np.random.seed(SEED)
print(f"[INFO] Parquet: {DEFAULT_PARQUET} | MAX_MONTHS={MAX_MONTHS} | USE_LAG1={USE_LAG1}")


In [ ]:

# 2) UTILS
def read_parquet_efficient(path, date_col):
    df = pd.read_parquet(path, engine="pyarrow")
    for c in df.select_dtypes(include=["int", "Int64"]).columns:
        df[c] = pd.to_numeric(df[c], downcast="integer")
    for c in df.select_dtypes(include=["float"]).columns:
        df[c] = pd.to_numeric(df[c], downcast="float")
    if date_col in df.columns:
        df[date_col] = pd.to_datetime(df[date_col], errors="coerce")
        df[date_col] = df[date_col].values.astype("datetime64[M]")
    for c in df.select_dtypes(include=["object", "string"]).columns:
        if df[c].nunique(dropna=True) < 5000 and df[c].nunique(dropna=True) / max(1,len(df)) < 0.5:
            df[c] = df[c].astype("category")
    if "cluster" in df.columns:
        if pd.api.types.is_numeric_dtype(df["cluster"]):
            df["cluster"] = pd.to_numeric(df["cluster"], downcast="integer")
        df["cluster"] = df["cluster"].astype("category")
    return df

def add_calendar_features_inplace(df, date_col):
    df["year"]  = df[date_col].dt.year.astype("int16")
    df["month"] = df[date_col].dt.month.astype("int8")
    df["quarter"] = df[date_col].dt.quarter.astype("int8")
    df["month_sin"] = np.sin(2*np.pi*df["month"]/12).astype("float32")
    df["month_cos"] = np.cos(2*np.pi*df["month"]/12).astype("float32")

def add_group_lag1_inplace(df, keys, date_col, target):
    df.sort_values([*keys, date_col], inplace=True, kind="stable")
    df["lag1"] = df.groupby(keys, observed=True)[target].shift(1).astype("float32")
    med = df.groupby(keys, observed=True)[target].transform("median")
    df["lag1"] = df["lag1"].fillna(med).fillna(df[target].median()).astype("float32")

def limit_to_last_n_months(df, date_col, n_months):
    months = sorted(df[date_col].dt.to_period("M").unique())
    if len(months) > n_months:
        keep = set(months[-n_months:])
        return df[df[date_col].dt.to_period("M").isin(keep)].reset_index(drop=True)
    return df

def rmse(y_true, y_pred):
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))


In [ ]:

# 3) LOAD + SHRINK + FEATURES
src = DEFAULT_PARQUET
assert Path(src).exists(), f"Parquet não encontrado: {src}"
df = read_parquet_efficient(src, DATE_COL)
if df[TARGET_COL].isna().any():
    df = df.loc[~df[TARGET_COL].isna()].reset_index(drop=True)
df = limit_to_last_n_months(df, DATE_COL, MAX_MONTHS)
for k in GRAIN_KEYS: assert k in df.columns, f"Ausente: {k}"
add_calendar_features_inplace(df, DATE_COL)
if USE_LAG1: add_group_lag1_inplace(df, GRAIN_KEYS, DATE_COL, TARGET_COL)
features = [c for c in df.columns if c not in {DATE_COL, TARGET_COL}]
print("[INFO] Features:", len(features))


In [ ]:

# 4) 1-FOLD TEMPORAL SPLIT (último mês)
all_months = sorted(df[DATE_COL].dt.to_period("M").unique())
val_month = all_months[-1]
train_mask = df[DATE_COL].dt.to_period("M") < val_month
val_mask   = df[DATE_COL].dt.to_period("M") == val_month
X_train, y_train = df.loc[train_mask, features], df.loc[train_mask, TARGET_COL].astype("float32")
X_val,   y_val   = df.loc[val_mask, features],   df.loc[val_mask, TARGET_COL].astype("float32")
print(f"[SPLIT] Train={X_train.shape} | Val={X_val.shape} | ValMonth={val_month}")


In [ ]:

# 5) MODELOS (Ridge + LightGBM)
cat_cols = list(df[features].select_dtypes(include=["category","object","string"]).columns)
num_cols = [c for c in features if c not in cat_cols]

ridge_pre = ColumnTransformer([
    ("num", StandardScaler(with_mean=False), num_cols),
    ("cat", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1), cat_cols)
], remainder="drop", sparse_threshold=0.3)

from sklearn.preprocessing import FunctionTransformer
def ensure_categories(X: pd.DataFrame) -> pd.DataFrame:
    X = X.copy()
    for c in cat_cols:
        if c in X.columns:
            X[c] = X[c].astype("category")
    return X
ensure_cat_transformer = FunctionTransformer(ensure_categories, validate=False)

ridge = Pipeline([("pre", ridge_pre), ("est", Ridge(alpha=1.0, random_state=SEED))])

lgbm = Pipeline([
    ("pre", ensure_cat_transformer),
    ("est", LGBMRegressor(
        objective="regression",
        learning_rate=0.12, n_estimators=600,
        num_leaves=31, min_child_samples=120,
        subsample=0.8, colsample_bytree=0.8,
        reg_lambda=1.0, reg_alpha=0.0,
        max_bin=191, n_jobs=2, random_state=SEED
    ))
])


In [ ]:

# 6) TREINO & AVALIAÇÃO
ridge.fit(X_train, y_train)
pred_ridge = ridge.predict(X_val)
rmse_ridge = rmse(y_val, pred_ridge)
print(f"[Ridge] RMSE val: {rmse_ridge:.4f}")

lgbm.fit(X_train, y_train, est__eval_set=[(X_val, y_val)], est__early_stopping_rounds=40, est__verbose=False)
best_iter = getattr(lgbm.named_steps["est"], "best_iteration_", None) or 300
pred_lgbm = lgbm.predict(X_val)
rmse_lgbm = rmse(y_val, pred_lgbm)
print(f"[LightGBM] RMSE val: {rmse_lgbm:.4f} | best_iter≈{best_iter}")

import pandas as pd
metrics_df = pd.DataFrame([
    {"model":"Ridge","rmse_val":rmse_ridge},
    {"model":"LightGBM","rmse_val":rmse_lgbm}
]).sort_values("rmse_val", ascending=True).reset_index(drop=True)
metrics_df


In [ ]:

# 7) VENCEDOR & SALVAR PIPELINE
winner = metrics_df.iloc[0]["model"]
print(f"[WINNER] {winner}")
if winner == "Ridge":
    final = ridge
else:
    final_rounds = int(min(max(150, best_iter), 400))
    final = Pipeline([
        ("pre", lgbm.named_steps["pre"]),
        ("est", lgbm.named_steps["est"].set_params(n_estimators=final_rounds))
    ])
final.fit(df[features], df[TARGET_COL].astype("float32"))

schema = {
    "date_col": DATE_COL, "target_col": TARGET_COL, "grain_keys": GRAIN_KEYS,
    "features": features, "categoricals": cat_cols, "winner_model": str(winner)
}
fallback = {
    "numeric_median": {c: float(df[c].median()) for c in df[features].select_dtypes(include=["number"]).columns},
    "categorical_default": "missing"
}
OUT_DIR = Path("./outputs"); OUT_DIR.mkdir(parents=True, exist_ok=True)
joblib.dump(final, OUT_DIR / "best_model_pipeline.pkl")
with open(OUT_DIR / "schema.json","w",encoding="utf-8") as f: json.dump(schema, f, ensure_ascii=False, indent=2)
with open(OUT_DIR / "fallback_stats.json","w",encoding="utf-8") as f: json.dump(fallback, f, ensure_ascii=False, indent=2)
print("[FILES] Salvos em ./outputs/")
